크롤링 시도

URL에 있는 상품번호가 패턴을 보이지 않아 일단 먼저 셀리니움으로 모든 상품 번호를 먼저 가져온다

***침실***

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20078", "침대"),          # 예시: 침대 카테고리
    ("20079", "매트릭스"),
    ("20080", "화장대, 서랍장")     # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 10 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_livingroom_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


***거실***

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20081", "소파"),          # 예시: 침대 카테고리
    ("20082", "테이블"),      # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 10 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_livingroom_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 2개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 소파 (ID: 20081)
-> [소파] 1 페이지 로딩 중...
✅ [소파] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [소파] 2 페이지 로딩 중...
✅ [소파] 2 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 3 페이지 로딩 중...
✅ [소파] 3 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [소파] 4 페이지 로딩 중...
✅ [소파] 4 페이지 완료. 수집된 ID 20개, 새로운 ID 18개 추가.
-> [소파] 5 페이지 로딩 중...
✅ [소파] 5 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 6 페이지 로딩 중...
✅ [소파] 6 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 7 페이지 로딩 중...
✅ [소파] 7 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 8 페이지 로딩 중...
✅ [소파] 8 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 9 페이지 로딩 중...
✅ [소파] 9 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 10 페이지 로딩 중...
✅ [소파] 10 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 11 페이지 로딩 중...
✅ [소파] 11 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [소파] 12 페이지 로딩 중...
✅ [소파] 12 페이지 완료. 수집된 ID 8개, 새로운 ID 8개 추가.
-> [소파] 13 페이지 로딩 중...
❌ [소파] 13 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHa

***다이닝***

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20083", "식탁"),          # 예시: 침대 카테고리
    ("20084", "식탁의자"),
    ("20085", "주방수납장")            # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 10 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_Kitchen_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 3개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 식탁 (ID: 20083)
-> [식탁] 1 페이지 로딩 중...
✅ [식탁] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [식탁] 2 페이지 로딩 중...
✅ [식탁] 2 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [식탁] 3 페이지 로딩 중...
✅ [식탁] 3 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [식탁] 4 페이지 로딩 중...
✅ [식탁] 4 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [식탁] 5 페이지 로딩 중...
✅ [식탁] 5 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [식탁] 6 페이지 로딩 중...
✅ [식탁] 6 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [식탁] 7 페이지 로딩 중...
✅ [식탁] 7 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [식탁] 8 페이지 로딩 중...
✅ [식탁] 8 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [식탁] 9 페이지 로딩 중...
✅ [식탁] 9 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [식탁] 10 페이지 로딩 중...
✅ [식탁] 10 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [식탁] 11 페이지 로딩 중...
✅ [식탁] 11 페이지 완료. 수집된 ID 14개, 새로운 ID 14개 추가.
-> [식탁] 12 페이지 로딩 중...
❌ [식탁] 12 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff627921eb5+80197]
	GetHandleVerifier [0x0x7ff6

***옷장***

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20086", "옷장"),          # 예시: 침대 카테고리
    ("20087", "드레스룸"),
    ("20088", "붙박이장"),
    ("20490", "수납,서랍장")             # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 10 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_Dressroom_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 4개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 옷장 (ID: 20086)
-> [옷장] 1 페이지 로딩 중...
✅ [옷장] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 17개 추가.
-> [옷장] 2 페이지 로딩 중...
✅ [옷장] 2 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [옷장] 3 페이지 로딩 중...
✅ [옷장] 3 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 4 페이지 로딩 중...
✅ [옷장] 4 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 5 페이지 로딩 중...
✅ [옷장] 5 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 6 페이지 로딩 중...
✅ [옷장] 6 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 7 페이지 로딩 중...
✅ [옷장] 7 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 8 페이지 로딩 중...
✅ [옷장] 8 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 9 페이지 로딩 중...
✅ [옷장] 9 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 10 페이지 로딩 중...
✅ [옷장] 10 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 11 페이지 로딩 중...
✅ [옷장] 11 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 12 페이지 로딩 중...
✅ [옷장] 12 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 13 페이지 로딩 중...
✅ [옷장] 13 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [옷장] 14 페이지 로딩 중

***키즈룸***

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20089", "수납장,책상"),          # 예시: 침대 카테고리
    ("20090", "침대"),
    ("20091", "옷장, 서랍장"),
    ("20092", "책상,의자")             # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_Kidsroom_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 4개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 수납장,책상 (ID: 20089)
-> [수납장,책상] 1 페이지 로딩 중...
✅ [수납장,책상] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 18개 추가.
-> [수납장,책상] 2 페이지 로딩 중...
✅ [수납장,책상] 2 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [수납장,책상] 3 페이지 로딩 중...
✅ [수납장,책상] 3 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 4 페이지 로딩 중...
✅ [수납장,책상] 4 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 5 페이지 로딩 중...
✅ [수납장,책상] 5 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 6 페이지 로딩 중...
✅ [수납장,책상] 6 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 7 페이지 로딩 중...
✅ [수납장,책상] 7 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 8 페이지 로딩 중...
✅ [수납장,책상] 8 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 9 페이지 로딩 중...
✅ [수납장,책상] 9 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 10 페이지 로딩 중...
✅ [수납장,책상] 10 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 11 페이지 로딩 중...
✅ [수납장,책상] 11 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [수납장,책상] 12 페이지 로딩 중...
✅ [수납장,책상] 12 페이지 완료. 수집된 ID 13개, 새로

***학생방***

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20094", "책상"),          # 예시: 침대 카테고리
    ("20095", "책상 수납장"),
    ("20096", "침대"),
    ("20097", "옷장,서랍장"),
    ("20098", "의자")               # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_Studentroom_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 5개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 책상 (ID: 20094)
-> [책상] 1 페이지 로딩 중...
✅ [책상] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 18개 추가.
-> [책상] 2 페이지 로딩 중...
✅ [책상] 2 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [책상] 3 페이지 로딩 중...
✅ [책상] 3 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [책상] 4 페이지 로딩 중...
✅ [책상] 4 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [책상] 5 페이지 로딩 중...
✅ [책상] 5 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [책상] 6 페이지 로딩 중...
✅ [책상] 6 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [책상] 7 페이지 로딩 중...
✅ [책상] 7 페이지 완료. 수집된 ID 2개, 새로운 ID 2개 추가.
-> [책상] 8 페이지 로딩 중...
❌ [책상] 8 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff627921eb5+80197]
	GetHandleVerifier [0x0x7ff627921f10+80288]
	(No symbol) [0x0x7ff6276a02fa]
	(No symbol) [0x0x7ff6276f7cd7]
	(No symbol) [0x0x7ff6276f7f9c]
	(No symbol) [0x0x7ff62774ba87]
	(No symbol) [0x0x7ff6277203bf]
	(No symbol) [0x0x7ff6277487fb]
	(No symbol) [0x0x7ff627720153]
	(No symbol) [0x0x7ff6276e8b02]


***홈오피스***

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20099", "책상"),          # 예시: 침대 카테고리
    ("20100", "책상 수납장"),
    ("20101", "의자")             # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_Homeoffice_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 3개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 책상 (ID: 20099)
-> [책상] 1 페이지 로딩 중...
✅ [책상] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 18개 추가.
-> [책상] 2 페이지 로딩 중...
✅ [책상] 2 페이지 완료. 수집된 ID 20개, 새로운 ID 19개 추가.
-> [책상] 3 페이지 로딩 중...
✅ [책상] 3 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [책상] 4 페이지 로딩 중...
✅ [책상] 4 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [책상] 5 페이지 로딩 중...
✅ [책상] 5 페이지 완료. 수집된 ID 18개, 새로운 ID 18개 추가.
-> [책상] 6 페이지 로딩 중...
❌ [책상] 6 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff627921eb5+80197]
	GetHandleVerifier [0x0x7ff627921f10+80288]
	(No symbol) [0x0x7ff6276a02fa]
	(No symbol) [0x0x7ff6276f7cd7]
	(No symbol) [0x0x7ff6276f7f9c]
	(No symbol) [0x0x7ff62774ba87]
	(No symbol) [0x0x7ff6277203bf]
	(No symbol) [0x0x7ff6277487fb]
	(No symbol) [0x0x7ff627720153]
	(No symbol) [0x0x7ff6276e8b02]
	(No symbol) [0x0x7ff6276e98d3]
	GetHandleVerifier [0x0x7ff627bde83d+2949837]
	GetHandleVerifier [0x0x7ff627bd8c6a+2926330]
	GetHa

***홈&데코***

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20102", "생활용품"),          # 예시: 침대 카테고리
    ("20103", "수납정리"),
    ("20104", "주방수납"),
    ("20105", "테이블웨어"),
    ("20106", "쿸웨어"),
    ("20107", "기타주방용품"),
    ("20108", "침구"),
    ("20109", "커튼"),
    ("20110", "홈패드릭"),
    ("20111", "조명"),
    ("20112", "가전"),
    ("20113", "인테리어소품"),
    ("20114", "한샘 AS부품")             # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url 
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_HomeDeco_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 13개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 생활용품 (ID: 20102)
-> [생활용품] 1 페이지 로딩 중...
✅ [생활용품] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [생활용품] 2 페이지 로딩 중...
✅ [생활용품] 2 페이지 완료. 수집된 ID 20개, 새로운 ID 18개 추가.
-> [생활용품] 3 페이지 로딩 중...
✅ [생활용품] 3 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [생활용품] 4 페이지 로딩 중...
✅ [생활용품] 4 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [생활용품] 5 페이지 로딩 중...
✅ [생활용품] 5 페이지 완료. 수집된 ID 9개, 새로운 ID 9개 추가.
-> [생활용품] 6 페이지 로딩 중...
❌ [생활용품] 6 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff627921eb5+80197]
	GetHandleVerifier [0x0x7ff627921f10+80288]
	(No symbol) [0x0x7ff6276a02fa]
	(No symbol) [0x0x7ff6276f7cd7]
	(No symbol) [0x0x7ff6276f7f9c]
	(No symbol) [0x0x7ff62774ba87]
	(No symbol) [0x0x7ff6277203bf]
	(No symbol) [0x0x7ff6277487fb]
	(No symbol) [0x0x7ff627720153]
	(No symbol) [0x0x7ff6276e8b02]
	(No symbol) [0x0x7ff6276e98d3]
	GetHandleVerifier [0x0x7ff627bde83d+2949837]
	GetHandleVerifier [0x0x7ff